<a href="https://colab.research.google.com/github/tpessoa-dev/plataformas-cognitivas/blob/main/Plataformas_Azure_Treino_Fora_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [2]:
!git clone https://github.com/elthonf/azure-ml.git

Cloning into 'azure-ml'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 105 (delta 41), reused 83 (delta 24), pack-reused 0
Receiving objects: 100% (105/105), 6.01 MiB | 22.06 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
!pip install azureml-core
!pip install scikit-learn==0.20.3

     |████████████████████████████████| 2.2 MB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 35.5 MB/s 
     |████████████████████████████████| 101 kB 9.5 MB/s 
     |████████████████████████████████| 85 kB 3.2 MB/s 
     |████████████████████████████████| 312 kB 68.1 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 3.0 MB 24.7 MB/s 
     |████████████████████████████████| 141 kB 53.0 MB/s 
     |████████████████████████████████| 40 kB 4.7 MB/s 
     |████████████████████████████████| 547 kB 47.3 MB/s 
     |████████████████████████████████| 796 kB 49.0 MB/s 
     |████████████████████████████████| 147 kB 39.1 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 165 kB 46.3 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
     |████████████████████████

In [1]:
!pip install azureml-dataprep
!pip install azureml-dataset-runtime

     |████████████████████████████████| 39.4 MB 13 kB/s 
     |████████████████████████████████| 10.5 MB 42.2 MB/s 
     |████████████████████████████████| 1.3 MB 44.4 MB/s 
     |████████████████████████████████| 28.7 MB 63 kB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 75 kB 3.5 MB/s 
     |████████████████████████████████| 39.4 MB 53 kB/s 
     |████████████████████████████████| 10.4 MB 22.4 MB/s 
  Attempting uninstall: azureml-dataprep-rslex
    Found existing installation: azureml-dataprep-rslex 1.20.1
    Uninstalling azureml-dataprep-rslex-1.20.1:
      Successfully uninstalled azureml-dataprep-rslex-1.20.1
  Attempting uninstall: azureml-dataprep
    Found existing installation: azureml-dataprep 2.22.2
    Uninstalling azureml-dataprep-2.22.2:
      Successfully uninstalled azureml-dataprep-2.22.2


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import sklearn
import math
from azureml.core import Experiment
import random
import pandas as pd
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
import uuid
# O azureml-core da versão 1.0.72 ou superior é requerido
# é necessário azureml-dataprep[pandas] na versão 1.1.34 ou superior
from azureml.core import Workspace, Dataset

In [3]:
from azureml.core import Model

In [4]:
def get_a_funnyName():
    f = open("azure-ml/datasets/names/funnynames.txt", "r")
    nomes = f.readlines()
    return random.choice(nomes).rstrip('\n')

# Conexão

In [5]:
#Trocar os códigos abaixo pelos da sua instância!
subscription_id = '273cbbc1-68d8-4126-8e7a-d9cd42ee6687'
resource_group = 'Aulas-FIAP'
workspace_name = 'mlstudio'

workspace = Workspace(subscription_id, resource_group, workspace_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EW3NDVS6H to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


# Carga de dados

In [6]:
mydf = pd.read_csv('azure-ml/datasets/statistical/BaseDefault01.csv')
mydf.head()

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade,default
0,"Simon, Rodriguez",4472.190323,42.036031,0,0,1,0,0,3,0
1,"Daniel, Castro",4592.774312,48.230662,1,0,1,0,1,2,0
2,"Myhue, Lin",2486.538807,56.881709,0,1,0,0,0,0,1
3,"Destiny, Richardson-Pacheco",2852.340117,51.684021,1,1,0,0,0,2,1
4,"Brittany, Cohen-Wilson",4703.782812,50.729078,1,1,1,0,1,2,0


In [7]:
dataset = Dataset.get_by_name(workspace, name='Risco-De-Credito')
df = dataset.to_pandas_dataframe()
df.head()

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade,default
0,"Simon, Rodriguez",4472.190323,42.036031,0,0,1,0,0,3,0
1,"Daniel, Castro",4592.774312,48.230662,1,0,1,0,1,2,0
2,"Myhue, Lin",2486.538807,56.881709,0,1,0,0,0,0,1
3,"Destiny, Richardson-Pacheco",2852.340117,51.684021,1,1,0,0,0,2,1
4,"Brittany, Cohen-Wilson",4703.782812,50.729078,1,1,1,0,1,2,0


# Treinamento

Identifica no dataset as variáveis independentes e a variavel alvo

In [8]:
targetcol = 'default'
y = mydf[targetcol]

Cria experimento

In [10]:
experiment = Experiment(workspace=workspace, name="Experimento_Banco")

## Treina modelo 01 (Classificador)

Cria uma execução para o experimento

In [20]:
run = experiment.start_logging(run_id= "Classificador1_" + str( uuid.uuid1() ) )
run.log("Tipo", "Classificador")

In [21]:
independentcols = ['renda', 'idade', 'etnia', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']
x = mydf[independentcols]
clf = rfc()
clf.fit(X=x, y=y)
clf.independentcols = independentcols

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Avaliações e Logs

In [22]:
clf_acuracia = clf.score(X=x, y=y)
print("Modelo 01 (classificador), criado com acurácia de: [{0}]".format(clf_acuracia))
run.log("acuracia", clf_acuracia)

Modelo 01 (classificador), criado com acurácia de: [0.98766]


In [23]:
#Demais logs (não serão comparados)
run.log("Versao sklearn", sklearn.__version__)
run.log("criterion", clf.criterion)
run.log("n_estimators", clf.n_estimators)
run.log("min_samples_leaf", clf.min_samples_leaf)
run.log("max_depth",  str(clf.max_depth))
run.log_list("Inputs", independentcols)

Exporta o modelo

In [24]:
model_name = "nome_arquivo.pkl"
filename = "outputs/" + model_name
joblib.dump(value=clf, filename=filename)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

['outputs/nome_arquivo.pkl']

Upload do arquivo, deve demorar um pouco

In [25]:
run.upload_file(name=model_name, path_or_stream=filename)

In [26]:
ls -l outputs

total 15880
-rw-r--r-- 1 root root 16257919 Aug 30 23:00 nome_arquivo.pkl


Finaliza o experimento

In [27]:
run.complete()

# Atividade 1 : Análise na Azure

Crie n modelos, e envie para a Azure. Em seguida, compare os resultados dentro da plataforma.  

Obs.:  
n = 8 $\times$ 4 $\times$ 3

In [ ]:
experiment = Experiment(workspace=workspace, name="Experimento_RiscoCredito")

In [ ]:
n_estimators = [1, 2, 3, 5, 10, 20, 30, 100]
max_depth = [1, 2, 3, None]
min_samples_leaf = [10, 5, 1]

In [ ]:
modelo=0
for a in n_estimators:
   for b in max_depth:
     for c in min_samples_leaf:
      modelo=modelo+1
      run = experiment.start_logging(run_id= "Classificador1_" + str( uuid.uuid1() ) )
      run.log("Tipo", "Classificador")
      independentcols = ['renda', 'idade', 'etnia', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']
      
      x = mydf[independentcols]
      clf = rfc(n_estimators=a,max_depth=b,min_samples_leaf=c)
      clf.fit(X=x, y=y)
      clf.independentcols = independentcols

      clf_acuracia = clf.score(X=x, y=y)
      print("Modelo ",modelo," (classificador), criado com acurácia de: [{0}]".format(clf_acuracia))
      run.log("acuracia", clf_acuracia)

      run.log("Versao sklearn", sklearn.__version__)
      run.log("criterion", clf.criterion)
      run.log("n_estimators", clf.n_estimators)
      run.log("min_samples_leaf", clf.min_samples_leaf)
      run.log("max_depth",  str(clf.max_depth))
      run.log_list("Inputs", independentcols)
      
      model_name = "nome_arquivo.pkl"
      filename = "outputs/" + model_name
      joblib.dump(value=clf, filename=filename)

      run.upload_file(name=model_name, path_or_stream=filename)
      run.complete()



Modelo  1  (classificador), criado com acurácia de: [0.75151]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo  2  (classificador), criado com acurácia de: [0.7497]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo  3  (classificador), criado com acurácia de: [0.66698]


...

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo  95  (classificador), criado com acurácia de: [0.91511]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo  96  (classificador), criado com acurácia de: [0.99997]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

# Atividade 2 : Regressor

Treinar um segundo modelo (Regressor sem a variavel etnia)

In [12]:
run = experiment.start_logging(run_id= "Regressor_" + str( uuid.uuid1() ) )
run.log("Tipo", "Regressor")

In [13]:
independentcols = ['renda', 'idade', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']
x = mydf[independentcols]
n1 = rfr()
n1.fit(X=x, y=y)
n1.independentcols = independentcols

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
n1_acuracia = n1.score(X=x, y=y)
print("Modelo 01 (regressor), criado com acurácia de: [{0}]".format(n1_acuracia))
run.log("acuracia", n1_acuracia)

Modelo 01 (regressor), criado com acurácia de: [0.9176523699446449]


In [15]:
#Demais logs (não serão comparados)
run.log("Versao sklearn", sklearn.__version__)
run.log("criterion", n1.criterion)
run.log("n_estimators", n1.n_estimators)
run.log("min_samples_leaf", n1.min_samples_leaf)
run.log("max_depth",  str(n1.max_depth))
run.log_list("Inputs", independentcols)

In [16]:
model_name = "nome_arquivo.pkl"
filename = "outputs/" + model_name
joblib.dump(value=n1, filename=filename)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

['outputs/nome_arquivo.pkl']

In [17]:
run.upload_file(name=model_name, path_or_stream=filename)

In [18]:
ls -l outputs

total 12380
-rw-r--r-- 1 root root 12676008 Aug 30 22:59 nome_arquivo.pkl


In [19]:
run.complete()

- Usar o classificador acima como modelo
- Gravar log de algumas variáveis
- Gravar log da acurácia
- Exportar o modelo em arquivo
- Enviar para AZURE Cloud

In [ ]:
# Upload direto
# modelo = Model.register(workspace=workspace, model_name="ModeloUploadDireto", 
#                        model_framework=Model.Framework.SCIKITLEARN,
#                        model_framework_version='0.20.3',
#                        model_path=filename)